In [ ]:
import os
import time
import random
import numpy as np
from pprint import pprint
import pandas as pd

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
# from tensorflow.keras.callbacks import ReduceLROnPlateau
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Resizing
from tensorflow.keras.layers import Rescaling
# from tensorflow.keras.models import load_model

from sklearn import metrics
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
from plotly.subplots import make_subplots

try:
    from PDSUtilities.tensorflow.tvp_split import tvp_split
    from PDSUtilities.tensorflow.plot_history import plot_history
    from PDSUtilities.plotly.create_image_subplots import create_image_subplots
    import PDSUtilities.plotly.templates
except ImportError as e:
    try:
        %pip install PDSUtilities --upgrade
        from PDSUtilities.tensorflow.tvp_split import tvp_split
        from PDSUtilities.tensorflow.plot_history import plot_history
        from PDSUtilities.plotly.create_image_subplots import create_image_subplots
        import PDSUtilities.plotly.templates
    except ImportError as e:
        raise ImportError("You must install PDSUtilities to plot importance and trees...") from e

from ipywidgets import HBox

# from utilities import read_or_create_csv

print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
print("Devices: ", tf.config.list_physical_devices("GPU"))

LABELS_COLUMN = "Label"
FILENAMES_COLUMN = "Filename"

BATCH_SIZE = 128
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
CHANNELS = 3

FILTERS = 32
KERNEL = 5
STRIDES = 2
ACTIVATION = "relu"
DROPOUT = 0.5
DENSE = 256
CLASSES = 29
LEARNING_RATE = 0.001

ROOT_DIR = "/kaggle/input/asl-alphabet"
ROOT_DIR = "../archive"
TRAINING_DIR = os.path.join(ROOT_DIR, "asl_alphabet_train/asl_alphabet_train")
TESTING_DIR = os.path.join(ROOT_DIR, "asl_alphabet_test/asl_alphabet_test")
# /kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/*_test.jpg
# /kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/?/?#.jpg
# 
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         if "_test" in dirname:
#             print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels = [label for label in os.listdir(TRAINING_DIR)]
df = pd.DataFrame([
    {
        LABELS_COLUMN: label,
        FILENAMES_COLUMN: os.path.join(TRAINING_DIR, label, filename)
    }
    for label in labels
    for filename in os.listdir(os.path.join(TRAINING_DIR, label))
])
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
dt, dv, dp = tvp_split(df, 0.2, 0.1)
print(f"Length of training images list: {len(dt)}.")
print(f"Length of validation images list: {len(dv)}.")
print(f"Length of prediction images list: {len(dp)}.")
print(f"Number of unique labels in training images list: {len(dt[LABELS_COLUMN].unique())}.")
print(f"Number of unique labels in validation images list: {len(dv[LABELS_COLUMN].unique())}.")
print(f"Number of unique labels in prediction images list: {len(dp[LABELS_COLUMN].unique())}.")
# Insert plot here showing distributions of labels across the three sets...

In [ ]:
# Do not scale images here...do that in a network layer...
xyt = ImageDataGenerator().flow_from_dataframe(dt, x_col=FILENAMES_COLUMN, y_col=LABELS_COLUMN,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), class_mode='categorical', batch_size=BATCH_SIZE
)
xyv = ImageDataGenerator().flow_from_dataframe(dv, x_col=FILENAMES_COLUMN, y_col=LABELS_COLUMN,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), class_mode='categorical', batch_size=BATCH_SIZE,
    shuffle=False
)
xyp = ImageDataGenerator().flow_from_dataframe(dp, x_col=FILENAMES_COLUMN, y_col=LABELS_COLUMN,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), class_mode='categorical', batch_size=BATCH_SIZE,
    shuffle=False
)

x, y = next(xyt)
class_labels = list(xyt.class_indices.keys())
class_labels = ["Label: " + class_labels[c] for c in np.argmax(y[0:16], axis=-1)]
fig = create_image_subplots(x[:16], labels=class_labels, width=720, height=800)
fig.show()

In [ ]:
# MaxAbsPool2D
import tensorflow as tf

class MaxAbsPool2D(tf.keras.layers.Layer):
    def __init__(self, pool_size, pad_to_fit=False):
        super(MaxAbsPool2D, self).__init__()
        self.pad = pad_to_fit
        self.pool_size = pool_size

    def compute_output_shape(self, in_shape):
        if self.pad:
            return (in_shape[0],
                    tf.math.ceil(in_shape[1] / self.pool_size),
                    tf.math.ceil(in_shape[2] / self.pool_size),
                    in_shape[3])
        return (in_shape[0],
                (in_shape[1] // self.pool_size),
                (in_shape[2] // self.pool_size),
                in_shape[3])

    def compute_padding(self, in_shape):
        mod_y = in_shape[1] % self.pool_size
        y1 = mod_y // 2
        y2 = mod_y - y1
        mod_x = in_shape[2] % self.pool_size
        x1 = mod_x // 2
        x2 = mod_x - x1
        self.padding = ((0, 0), (y1, y2), (x1, x2), (0, 0))

    def build(self, input_shape):
        self.in_shape = input_shape
        self.out_shape = self.compute_output_shape(self.in_shape)
        self.compute_padding(self.in_shape)

    def stack(self, inputs):
        if self.pad:
            inputs = tf.pad(inputs, self.padding)
        max_height = (tf.shape(inputs)[1] // self.pool_size) * self.pool_size
        max_width = (tf.shape(inputs)[2] // self.pool_size) * self.pool_size
        stack = tf.stack(
            [inputs[:, i:max_height:self.pool_size, j:max_width:self.pool_size, :]
             for i in range(self.pool_size) for j in range(self.pool_size)],
            axis=-1)
        return stack

    @tf.function
    def call(self, inputs, training=None):
        stacked = self.stack(inputs)
        inds = tf.argmax(tf.abs(stacked), axis=-1, output_type=tf.int32)
        ks = tf.shape(stacked)
        idx = tf.stack([
            *tf.meshgrid(
                tf.range(0, ks[0]),
                tf.range(0, ks[1]),
                tf.range(0, ks[2]),
                tf.range(0, ks[3]),
                indexing='ij'
            ), inds],
            axis=-1)
        x = tf.gather_nd(stacked, idx)
        x = tf.reshape(x, (-1, *self.out_shape[1:]))
        return x

In [ ]:
# CosSim2D
import math

import tensorflow as tf

class CosSim2D(tf.keras.layers.Layer):
    def __init__(self, kernel_size, units=32, stride=1, padding='valid'):
        super(CosSim2D, self).__init__()
        self.units = units
        assert kernel_size in [1, 3, 5], "kernel of this size not supported"
        self.kernel_size = kernel_size
        if self.kernel_size == 1:
            self.stack = lambda x: x
        elif self.kernel_size == 3:
            self.stack = self.stack3x3
        elif self.kernel_size == 5:
            self.stack = self.stack5x5
        self.stride = stride
        if padding == 'same':
            self.pad = self.kernel_size // 2
            self.pad_1 = 1
            self.clip = 0
        elif padding == 'valid':
            self.pad = 0
            self.pad_1 = 0
            self.clip = self.kernel_size // 2

    def build(self, input_shape):
        self.in_shape = input_shape
        self.out_y = math.ceil((self.in_shape[1] - 2 * self.clip) / self.stride)
        self.out_x = math.ceil((self.in_shape[2] - 2 * self.clip) / self.stride)
        self.flat_size = self.out_x * self.out_y
        self.channels = self.in_shape[3]
        self.w = self.add_weight(
            shape=(1, self.channels * tf.square(self.kernel_size), self.units),
            initializer="glorot_uniform", name='w',
            trainable=True,
        )
        p_init = tf.constant_initializer(value=100**0.5)
        self.p = self.add_weight(
            shape=(self.units,), initializer=p_init, trainable=True, name='p')

        q_init = tf.constant_initializer(value=10**0.5)
        self.q = self.add_weight(
            shape=(1,), initializer=q_init, trainable=True, name='q')

    def l2_normal(self, x, axis=None, epsilon=1e-12):
        square_sum = tf.reduce_sum(tf.square(x), axis, keepdims=True)
        x_inv_norm = tf.sqrt(tf.maximum(square_sum, epsilon))
        return x_inv_norm

    def stack3x3(self, image):
        '''
            sliding window implementation for 3x3 kernel
        '''
        x = tf.shape(image)[2]
        y = tf.shape(image)[1]
        stack = tf.stack(
            [
                tf.pad(  # top row
                    image[:, :y - 1 - self.clip:, :x - 1 - self.clip, :],
                    tf.constant([[0, 0], [self.pad, 0], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, :y - 1 - self.clip, self.clip:x - self.clip, :],
                    tf.constant([[0, 0], [self.pad, 0], [0, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, :y - 1 - self.clip, 1 + self.clip:, :],
                    tf.constant([[0, 0], [self.pad, 0], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],

                tf.pad(  # middle row
                    image[:, self.clip:y - self.clip, :x - 1 - self.clip, :],
                    tf.constant([[0, 0], [0, 0], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                image[:, self.clip:y - self.clip:self.stride, self.clip:x - self.clip:self.stride, :],
                tf.pad(
                    image[:, self.clip:y - self.clip, 1 + self.clip:, :],
                    tf.constant([[0, 0], [0, 0], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],

                tf.pad(  # bottom row
                    image[:, 1 + self.clip:, :x - 1 - self.clip, :],
                    tf.constant([[0, 0], [0, self.pad], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1 + self.clip:, self.clip:x - self.clip, :],
                    tf.constant([[0, 0], [0, self.pad], [0, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1 + self.clip:, 1 + self.clip:, :],
                    tf.constant([[0, 0], [0, self.pad], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :]
            ], axis=3)
        return stack

    def stack5x5(self, image):
        '''
            sliding window implementation for 5x5 kernel
        '''
        x = tf.shape(image)[2]
        y = tf.shape(image)[1]
        stack = tf.stack(
            [
                tf.pad(  # top row
                    image[:, :y - 2 - self.clip:, :x - 2 - self.clip, :],
                    tf.constant([[0, 0], [self.pad, 0], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, :y - 2 - self.clip:, 1:x - 1 - self.clip, :],
                    tf.constant([[0, 0], [self.pad, 0], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, :y - 2 - self.clip:, self.clip:x - self.clip, :],
                    tf.constant([[0, 0], [self.pad, 0], [0, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, :y - 2 - self.clip:, 1 + self.clip:-1, :],
                    tf.constant([[0, 0], [self.pad, 0], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, :y - 2 - self.clip:, 2 + self.clip:, :],
                    tf.constant([[0, 0], [self.pad, 0], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],

                tf.pad(  # 2nd row
                    image[:, 1:y - 1 - self.clip:, :x - 2 - self.clip, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1:y - 1 - self.clip:, 1:x - 1 - self.clip, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1:y - 1 - self.clip:, self.clip:x - self.clip, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [0, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1:y - 1 - self.clip:, 1 + self.clip:-1, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1:y - 1 - self.clip:, 2 + self.clip:, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],

                tf.pad(  # 3rd row
                    image[:, self.clip:y - self.clip, :x - 2 - self.clip, :],
                    tf.constant([[0, 0], [0, 0], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, self.clip:y - self.clip, 1:x - 1 - self.clip, :],
                    tf.constant([[0, 0], [0, 0], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                image[:, self.clip:y - self.clip, self.clip:x - self.clip, :][:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, self.clip:y - self.clip, 1 + self.clip:-1, :],
                    tf.constant([[0, 0], [0, 0], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, self.clip:y - self.clip, 2 + self.clip:, :],
                    tf.constant([[0, 0], [0, 0], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],

                tf.pad(  # 4th row
                    image[:, 1 + self.clip:-1, :x - 2 - self.clip, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1 + self.clip:-1, 1:x - 1 - self.clip, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1 + self.clip:-1, self.clip:x - self.clip, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [0, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1 + self.clip:-1, 1 + self.clip:-1, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 1 + self.clip:-1, 2 + self.clip:, :],
                    tf.constant([[0, 0], [self.pad_1, self.pad_1], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],

                tf.pad(  # 5th row
                    image[:, 2 + self.clip:, :x - 2 - self.clip, :],
                    tf.constant([[0, 0], [0, self.pad], [self.pad, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 2 + self.clip:, 1:x - 1 - self.clip, :],
                    tf.constant([[0, 0], [0, self.pad], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 2 + self.clip:, self.clip:x - self.clip, :],
                    tf.constant([[0, 0], [0, self.pad], [0, 0], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 2 + self.clip:, 1 + self.clip:-1, :],
                    tf.constant([[0, 0], [0, self.pad], [self.pad_1, self.pad_1], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
                tf.pad(
                    image[:, 2 + self.clip:, 2 + self.clip:, :],
                    tf.constant([[0, 0], [0, self.pad], [0, self.pad], [0, 0]])
                )[:, ::self.stride, ::self.stride, :],
            ], axis=3)
        return stack

    @tf.function
    def call(self, inputs, training=None):
        channels = tf.shape(inputs)[-1]
        x = self.stack(inputs)
        x = tf.reshape(x, (-1, self.flat_size, channels * tf.square(self.kernel_size)))
        x_norm = (self.l2_normal(x, axis=2) + tf.square(self.q)/10)
        w_norm = (self.l2_normal(self.w, axis=1) + tf.square(self.q)/10)
        x = tf.matmul(x / x_norm, self.w / w_norm)
        sign = tf.sign(x)
        x = tf.abs(x) + 1e-12
        x = tf.pow(x, tf.square(self.p)/100)
        x = sign * x
        x = tf.reshape(x, (-1, self.out_y, self.out_x, self.units))
        return x


In [ ]:
MODEL = "SCS"

In [ ]:
if MODEL == "SCS":
    model = Sequential([
        InputLayer((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS), name="input"),
        #
        Rescaling(1.0/255.0, name="layer00"),
        #
        CosSim2D(KERNEL, FILTERS*1, padding="same"),
        MaxAbsPool2D(STRIDES, True),
        # Dropout(DROPOUT, name="layer04"),
        #
        CosSim2D(KERNEL, FILTERS*2, padding="same"),
        MaxAbsPool2D(STRIDES, True),
        # Dropout(DROPOUT, name="layer08"),
        #
        CosSim2D(KERNEL, FILTERS*2, padding="same"),
        MaxAbsPool2D(STRIDES, True),
        # Dropout(DROPOUT, name="layer12"),
        #
        CosSim2D(KERNEL, FILTERS*2, padding="same"),
        MaxAbsPool2D(STRIDES, True),
        # Dropout(DROPOUT, name="layer13"),
        #
        CosSim2D(KERNEL, FILTERS*4, padding="same"),
        MaxAbsPool2D(STRIDES, True),
        #
        Flatten(name="layer16"),
        Dense(DENSE, name="layer17"),
        Activation(ACTIVATION, name="layer18"),
        Dense(CLASSES, name="layer19"),
        Activation("softmax", name="layer20")
    ], name="ASL")
else:
    model = Sequential([
        InputLayer((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS), name="input"),
        #
        Rescaling(1.0/255.0, name="layer00"),
        #
        Conv2D(FILTERS*1, KERNEL, padding="same", name="layer01"),
        Activation(ACTIVATION, name="layer02"),
        MaxPool2D(strides=STRIDES, name="layer03"),
        # Dropout(DROPOUT, name="layer04"),
        #
        Conv2D(FILTERS*2, KERNEL, padding="same", name="layer05"),
        Activation(ACTIVATION, name="layer06"),
        MaxPool2D(strides=STRIDES, name="layer07"),
        # Dropout(DROPOUT, name="layer08"),
        #
        Conv2D(FILTERS*2, KERNEL, padding="same", name="layer09"),
        Activation(ACTIVATION, name="layer10"),
        MaxPool2D(strides=STRIDES, name="layer11"),
        # Dropout(DROPOUT, name="layer12"),
        #
        Conv2D(FILTERS*2, KERNEL, padding="same", name="layer09a"),
        Activation(ACTIVATION, name="layer10a"),
        MaxPool2D(strides=STRIDES, name="layer11a"),
        # Dropout(DROPOUT, name="layer12"),
        #
        Conv2D(FILTERS*4, KERNEL, padding="same", name="layer13"),
        Activation(ACTIVATION, name="layer14"),
        MaxPool2D(strides=STRIDES, name="layer15"),
        #
        Flatten(name="layer16"),
        Dense(DENSE, name="layer17"),
        Activation(ACTIVATION, name="layer18"),
        Dense(CLASSES, name="layer19"),
        Activation("softmax", name="layer20")
    ], name="ASL")

model.summary()


In [ ]:
optimizer = Adam(learning_rate=LEARNING_RATE)
loss = "categorical_crossentropy"
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
EPOCHS = 10
VERBOSE = 1
callbacks = [
    LambdaCallback(
        on_epoch_end=lambda batch, logs: time.sleep(15),
    )
]
history = model.fit(xyt, validation_data=xyv, epochs=EPOCHS, callbacks=callbacks, verbose=VERBOSE)

In [ ]:
plot_history(history)

In [ ]:

x, y = next(xyp)
p = model.predict(x, verbose=1)
p_labels = list(xyp.class_indices.keys())
p_labels = ["Label: " + p_labels[c] for c in np.argmax(p[0:16], axis=-1)]
fig = create_image_subplots(x[:16], labels=p_labels, width=720, height=800)
fig.show()

In [ ]:
p = model.predict(xyp, verbose=1)
report = metrics.classification_report(
    xyp.classes,
    np.argmax(p, axis=-1),
    target_names=list(xyp.class_indices.keys())
)
print(report)

In [ ]:
import plotly.express as px

def plot_confusion_matrix(confusion_matrix, labels):
    fig = go.Figure()
    fig.add_trace(
        go.Heatmap(
            z = confusion_matrix,
            x = labels,
            y = labels,
        )
    )
    return fig

In [ ]:
cm = metrics.confusion_matrix(xyp.classes, np.argmax(p, axis=-1))
fig = plot_confusion_matrix(cm, list(xyp.class_indices.keys()))
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
from keras import backend as K
print(model.optimizer.learning_rate.numpy())
K.set_value(model.optimizer.learning_rate, 0.0001)
print(model.optimizer.learning_rate.numpy())